In [1]:
import Robogame as rg
import networkx as nx
import altair as alt
import time, json
import pandas as pd
import numpy as np
from multiprocessing.dummy import Process

In [94]:
game = rg.Robogame("bob") 
# game = rg.Robogame("bob",server="12.12.31.1",port=2324)
game.setReady()

{'Error': 'Game already started'}

# Init setting

In [95]:
# tell the server we're ready to go
game = rg.Robogame("bob") # game = rg.Robogame("bob",server="12.12.31.1",port=2324)
game.setReady()
info = dict()
partInterest = list()
robotInterest = list()

def gather_info():
    while True:
        gameTime = game.getGameTime()
        if "Error" in gameTime:
            info["time"] = {"curtime":100, "unitsleft":0}
            return
        try:
            info["time"] = pd.DataFrame(pd.Series(gameTime)[["curtime", "unitsleft"]]).T
            game.setPartInterest(partInterest) # Set part interest
            game.setRobotInterest(robotInterest) # Set robot interest
            hints = game.getHints()
            partHints = pd.DataFrame(game.getAllPartHints()).drop_duplicates().pivot(
                columns="column", index="id", values="value"
            )
            predHints = pd.DataFrame(game.getAllPredictionHints()).set_index(["id"])
            all_robots = game.getRobotInfo().merge(partHints, left_index=True, right_index=True)
            all_robots["Part Hint Count"] = all_robots.count(axis=1) - 6
            info["All Robots"] = all_robots
            curtime = float(info["time"]["curtime"])
            robots = info["All Robots"].copy() # set the robot dataframe
            info["robots"] = robots[robots["expires"]>=curtime].sort_values(["expires", "Productivity"])
            info["num"] = info["robots"].merge(
                predHints, left_index=True, right_index=True
            ).sort_values(["expires", "Productivity"])
        except Exception:
            pass
thread = Process(target=gather_info)
thread.start()

# Source code for the visualization

In [7]:
# Show the priority to take a guess
def vis_priority():
    df = info["All Robots"].fillna(0)
    chart = alt.Chart(df, width=750, height=250).mark_circle().encode(
        x=alt.X("expires:Q", scale=alt.Scale(domain=[0, 100])),
        y=alt.Y("Part Hint Count:Q", scale=alt.Scale(domain=[-1, 11])),
        color=alt.Color("Productivity:Q", scale=alt.Scale(scheme="redyellowblue", reverse=True, domain=[-100, 100])),
        size=alt.value(50),
        tooltip=["id:N", "expires:Q", "Part Hint Count:Q", "Productivity:Q"]
    )
    time_df = pd.DataFrame({"time":[float(info["time"]["curtime"])]})
    timeline = alt.Chart(time_df).mark_rule().encode(
        x = alt.X("time:Q"),
        size= alt.value(1),
        tooltip=["time:Q"]
    )
    return (chart + timeline).interactive()

In [98]:
robots = game.getRobotInfo()
robots[~robots["Productivity"].isna()].drop(["winner"], axis=1)

,id,name,expires,Productivity,bets
16,16,Watervator Botson Jr.,22.0,83.013923,-1
37,37,Waterwidget Botson,21.0,-25.352515,-1
79,79,Excaborer Botchik I,21.0,-38.590951,-1
98,98,Thumpdigger Botchik II,22.0,36.154221,-1
99,99,Rockdigger Botski,21.0,73.620785,-1


Exception in thread Thread-9:
Traceback (most recent call last):
  File "/home/panda/.local/lib/python3.8/site-packages/urllib3/connection.py", line 156, in _new_conn
    conn = connection.create_connection(
  File "/home/panda/.local/lib/python3.8/site-packages/urllib3/util/connection.py", line 84, in create_connection
    raise err
  File "/home/panda/.local/lib/python3.8/site-packages/urllib3/util/connection.py", line 74, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/panda/.local/lib/python3.8/site-packages/urllib3/connectionpool.py", line 665, in urlopen
    httplib_response = self._make_request(
  File "/home/panda/.local/lib/python3.8/site-packages/urllib3/connectionpool.py", line 387, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/home/panda/anaconda3/envs/SI649/lib/python3.8/http/

In [91]:
robots = game.getRobotInfo()

part_df = pd.DataFrame(game.getAllPartHints()).drop_duplicates().pivot(
                columns="column", index="id", values="value"
)
prod_bots = robots[~robots["Productivity"].isna()].drop(["winner"], axis=1)
fields = ['Arakyd Vocabulator Model', 'Axial Piston Model', 'Nanochip Model']
robots = part_df[fields]
robots = robots.merge(prod_bots["Productivity"], left_index=True, right_index=True).reset_index()
charts = dict()
width=250
height=250
for field in fields:
    y= alt.Y("{}:N".format(field))
    heat = alt.Chart(robots, width=width, height=height).mark_rect().encode(
        x=alt.X("Productivity:Q", bin=True),
        y=y,
        color=alt.Color("count(*):Q")
    )
    dots = alt.Chart(robots, width=width, height=height).mark_circle(color="red").encode(
        x=alt.X("Productivity:Q"),
        y=y,
        tooltip=["Productivity:Q", "id:N"]
    )
    text = alt.Chart(robots).mark_text(dy=-10).encode(
        x=alt.X("Productivity:Q", bin=True),
        y=y,
        size=alt.value(15),
        text=alt.Text("count(*):Q")
    )
    charts[field] = heat + dots + text
chart = charts[fields[0]] | charts[fields[1]] | charts[fields[0]]
chart

alt.HConcatChart(...)

In [64]:
# Implementation of heatmap visualization
def trait_heatmap(field):
    height, width = 250, 500
    df = info["All Robots"]
    
    if field not in df.columns:
        return "We have not got any hints about {}".format(field)
    df = df[[field, "Productivity", "id"]].dropna()
    if field in ["Arakyd Vocabulator Model", "Axial Piston Model", "Nanochip Model"]:
        y= alt.Y("{}:N".format(field))
    else:
        y = alt.Y("{}:Q".format(field), bin=True)
    heat = alt.Chart(df, width=width, height=height).mark_rect().encode(
        x=alt.X("Productivity:Q", bin=True),
        y=y,
        color=alt.Color("count(*):Q")
    )
    dots = alt.Chart(df, width=width, height=height).mark_circle(color="red").encode(
        x=alt.X("Productivity:Q"),
        y=y,
        tooltip=["Productivity:Q", "id:N"]
    )
    text = alt.Chart(df).mark_text(dy=-10).encode(
        x=alt.X("Productivity:Q", bin=True),
        y=y,
        size=alt.value(15),
        text=alt.Text("count(*):Q")
    )
    return  heat + text + dots
trait_heatmap("Arakyd Vocabulator Model")

alt.LayerChart(...)

In [10]:
def trait_corr():
    df = info["All Robots"]
    df_corr = pd.DataFrame(columns=["Corr", "Num"])
    for column in ["Astrogation Buffer Length", "InfoCore Size", "AutoTerrain Tread Count", "Polarity Sinks", "Cranial Uplink Bandwidth", "Repulsorlift Motor HP", "Sonoreceptors"]:
        data = df[["Productivity", column]].dropna()
        corr = data["Productivity"].corr(data[column].astype(float))
        df_corr.loc[column] = [corr, len(data)]
    df_corr.fillna(0)
    df_corr.index.name="Traits"
    chart = alt.Chart(df_corr.reset_index(), width=300, height=300).mark_bar().encode(
        x=alt.X("Corr"),
        y=alt.Y("Traits:N", sort=alt.EncodingSortField(field='Corr', order='descending')),
        color=alt.Color("Num:Q"),
        tooltip=["Corr:Q","Num:Q"]
    )
    return chart

In [45]:
def number_hint(robots=[]):
    if not robots: robots = info["robots"].iloc[:10]["id"].tolist()
    df = info["num"]
    df = df[df["id"].isin(robots)]
    line = alt.Chart(df).mark_line().encode(
        x=alt.X("time:Q", scale=alt.Scale(domain=[0, 100])),
        y=alt.Y("value:Q"),
        color=alt.Color("id:N", scale=alt.Scale(domain=list(range(0, 100))))
    )
    circle = alt.Chart(df).mark_circle().encode(
        x=alt.X("time:Q"),
        y=alt.Y("value:Q"),
        color=alt.Color("id:N", scale=alt.Scale(domain=list(range(0, 100)))),
        tooltip=["time:Q", alt.Text("value:Q", format=".2f"), "id:N"]
    )
    time_df = pd.DataFrame({"time":[float(info["time"]["curtime"])]})
    timeline = alt.Chart(time_df).mark_rule().encode(
        x = alt.X("time:Q"),
        size= alt.value(1),
        tooltip=["time:Q"]
    )
    return (line + circle + timeline).resolve_scale(x="shared")

# Play the game
### 1. Query the time for the game

In [14]:
info["time"]

,curtime,unitsleft
0,14.62,85.38


### 2. Visualize the Priority

In [25]:
vis_priority()

alt.LayerChart(...)

In [40]:
info["robots"].iloc[:10]

,id,name,expires,winner,Productivity,bets,Arakyd Vocabulator Model,Astrogation Buffer Length,AutoTerrain Tread Count,Axial Piston Model,Cranial Uplink Bandwidth,InfoCore Size,Nanochip Model,Polarity Sinks,Repulsorlift Motor HP,Sonoreceptors,Part Hint Count
7,7,Pullborer Botchik II,98.0,2,47.245915,-1,model D,NaN,NaN,NaN,2.0,1.039138,NaN,0.264253,NaN,NaN,4
59,59,Fireminer Botberg,98.0,2,48.494736,-1,NaN,NaN,1.164531,NaN,45.0,1.122755,v2.0,0.382717,21.219077,NaN,6
14,14,Stonepulverizer Botski Jr.,99.0,-2,NaN,-1,model A,NaN,NaN,NaN,NaN,1.582571,windows 2000,NaN,NaN,99.0,3


### 3. Show the correlation between quantitative traits with productivity

In [26]:
trait_corr()

alt.Chart(...)

### 4. Set the interest for hint
| Part Hint                 | Type |
| ------------------------- | ---- |
| Astrogation Buffer Length | Q    |
| InfoCore Size             | Q    |
| AutoTerrain Tread Count   | Q    |
| Polarity Sinks            | Q    |
| Cranial Uplink Bandwidth  | Q    |
| Repulsorlift Motor HP     | Q    |
| Sonoreceptors             | Q    |
| Arakyd Vocabulator Model  | N    |
| Axial Piston Model        | N    |
| Nanochip Model            | N    |

In [159]:
partInterest = list("Astrogation Buffer Length") # Set the part for hints
robotInterest = list() # Set the robot if for hints

In [24]:
trait_heatmap("Sonoreceptors")

alt.VConcatChart(...)

### 5. Make Productivity predictions

### 6. Show the robot's number hint

In [47]:
number_hint()

alt.LayerChart(...)

### 7. Set bets

In [106]:
game.setBets({57:98})

{'Result': 'OK'}

In [36]:
info["All Robots"]

,id,name,expires,winner,Productivity,bets,Arakyd Vocabulator Model,Astrogation Buffer Length,AutoTerrain Tread Count,Axial Piston Model,Cranial Uplink Bandwidth,InfoCore Size,Nanochip Model,Polarity Sinks,Repulsorlift Motor HP,Sonoreceptors,Part Hint Count
0,0,Extraminer Boterson,75.0,-2,NaN,99,NaN,NaN,NaN,NaN,NaN,NaN,windows 2000,0.84536,NaN,84.0,2
1,1,Pushwidget Botchik,54.0,-2,NaN,-1,NaN,-0.053492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2,Wallminer Botberg Jr.,29.0,2,35.740069,-1,model A,NaN,NaN,NaN,NaN,NaN,NaN,0.744436,NaN,NaN,2
3,3,Extrahauler Boterson II,44.0,-2,NaN,-1,NaN,NaN,NaN,NaN,34.0,NaN,NaN,NaN,NaN,40.0,1
4,4,Wallminer Botberg I,28.0,2,88.397152,-1,NaN,-0.30119,1.467715,NaN,27.0,NaN,NaN,0.787552,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,Waterborer Botski,54.0,-2,NaN,-1,NaN,NaN,NaN,beta,NaN,NaN,NaN,NaN,-99.916325,17.0,2
96,96,Excawalker Botsky I,93.0,-2,NaN,-1,NaN,NaN,NaN,NaN,50.0,NaN,NaN,0.339969,-60.181329,NaN,2
97,97,Rocklift Botchik,80.0,-2,NaN,-1,model A,NaN,NaN,beta,NaN,NaN,windows 2000,NaN,-99.990359,NaN,3
98,98,Thumpdigger Botchik II,22.0,2,36.154221,-1,NaN,NaN,NaN,beta,NaN,NaN,NaN,NaN,NaN,NaN,1
